In [2]:
import json
import re
import requests
from pprintpp import pprint as pp
import ipywidgets as widgets
from IPython.core.display import display, HTML
import time
import IPython
from IPython.display import Javascript, display
import ipywidgets as widgets
from ipywidgets import interact




display(HTML("""
<script> function code_toggle() {
    if (code_shown){
        alert('if');
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
    alert('else');
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  };

  $(document).ready(function(){
    code_shown=false;
    alert('loaded');
    $('div.input').hide()  
  });
  
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
"""))


# Custom Student Activity Report
<hr>
This is a dashboard to generate a csv of a particular student's page_views activity given a window of time. 


There are 4 variables which must be provided by the person running the program.

1. you api_key - description of how this is confidential
2. the students user_id ( when you go the their canvas account page the info that is the user is: `https://canvas.upenn.edu/accounts/<account_id>/users/<user_id>`
3. the start date you want to start collecting  
4. the end date you want to stop collecting

NOTE: time in canvas is in military UTC time so add 12 and then 5 hrs to the EST you want to query 

note to maddy: remember that the api get request you want to do data = {...} and not files = {...}

something like this works!
https://upenn.test.instructure.com/api/v1/users/5389365/page_views?start_time=2018-11-19&end_time=2018-11-21&page=first&per_page=10

In [35]:
# time converter: takes EST time and formats it in EST 
# ( Coordinated Universal Time is 5 hours ahead of Eastern Time )
from datetime import datetime
from pytz import timezone

##### edit #######
date_str = "2018-11-12 18:00:00" # in military time

####### do not edit ##########
fmt = "%Y-%m-%d %H:%M:%S"
datetime_obj = datetime.strptime(date_str, fmt)
datetime_obj_est = datetime_obj.replace(tzinfo=timezone('US/Eastern'))
now_utc = datetime_obj_est.astimezone(timezone('UTC'))
print("entered EST time:", datetime_obj_est.strftime(fmt+ " %Z%z"))
print("time in UTC:\t ", now_utc.strftime(fmt + " %Z%z"))

entered EST time: 2018-11-12 18:00:00 LMT-0456
time in UTC:	  2018-11-12 22:56:00 UTC+0000


In [1]:
import json
import re
import requests
from pprintpp import pprint as pp
import ipywidgets as widgets
from IPython.core.display import display, HTML
import time
import IPython
from IPython.display import Javascript, display
import ipywidgets as widgets
from ipywidgets import interact

api = widgets.Text(value='<API KEY>', disabled=False)
s_id = widgets.Text(value='<Student ID>', disabled=False)
st = widgets.Text(value='<Start Time>', disabled=False)
en = widgets.Text(value='<End Time>', disabled=False)
print("please fill in the following cells with your information and then run the following cell")
display(api, s_id, st, en)


please fill in the following cells with your information and then run the following cell


Text(value='<API KEY>')

Text(value='<Student ID>')

Text(value='<Start Time>')

Text(value='<End Time>')

In [10]:
# button to run cells
from IPython.display import Javascript, display
from ipywidgets import widgets

"""button = widgets.Button(description="Create next input")
display(button)

def run_cell():"""
    # just some important imports ,, run this cell when u start 
#5389365
print("hi")
try:
    assert(api.value != '<API KEY>')
    assert(s_id.value != '<Student ID>')
    assert(st.value != '<Start Time>')
    assert(en.value != '<End Time>')
except AssertionError:
    print('Seems like one of the parameters were not set...')
    raise

api_key = api.value #if you are unsure about this ask maddy hodges because this is very confidential 
student_id = s_id.value  # see description above i.e. # "5443278"
start = st.value      # i.e. 2018-11-19
end = en.value        # i.e. 2018-11-20      
                 # this will collect time from 11-19T00:00 UTC until 11-20T00:00 UTC 
                 # equivalent EST military time is 11-18T19:00 until 11-19T19:00

domain = 'https://canvas.upenn.edu'#'https://upenn.test.instructure.com'
headers = {
    'Authorization': 'Bearer %s' % (api_key) ,
}
#student_id = '5389365' # confirmed correct id
url = '%s/api/v1/users/%s' % (domain,student_id)
data = {
    'start_time':start,
    'end_time':end
}
response = requests.get(url,headers=headers)
r = response.json()
print("Querying the URL: ", url)
print("Looking at student: %s, %s " % (r['name'],r['email']))
print("From %s to %s UTC" % (start, end))
print("If this is not the correct info please do not continue")

    

#button.on_click(run_cell())



hi
Querying the URL:  https://canvas.upenn.edu/api/v1/users/5482827
Looking at student: Michele Meline, meline@sas.upenn.edu 
From 2018-12-01 to 2018-12-10 UTC
If this is not the correct info please do not continue


In [11]:
#https://upenn.test.instructure.com/api/v1/users/5389365/page_views?start_time=2018-11-19&end_time=2018-11-21&page=first&per_page=10
import csv,sys

url ='%s/api/v1/users/%s/page_views?start_time=%s&end_time=%s&per_page=100' % (domain,student_id,start,end)
student_id = '5443278'

data = {
    'start_time': '2018-11-19',
    'end_time': '2018-11-20'  
}
#url ='%s/api/v1/users/%s/page_views' % (domain,student_id)
response = requests.get(url,headers=headers)
r = response.json()
pp(response.headers)
parsed_links = requests.utils.parse_header_links(response.headers['link'].rstrip('>').replace('>,<', ',<'))
pp(parsed_links)#response.headers['Link'])

display(HTML('<p><b>queried:</b> %s</p><p><b>result(%s):</b></p>' % (url,len(r))))
pp(r)



outputFile = 'Student_Activity.csv'
data = r
pp(data[0].keys())
output = csv.writer(open(outputFile,"w", newline='')) #create a csv.write
output.writerow(['session_id', 'url', 'context_type', 'asset_type', 'controller', 'action', 'interaction_seconds', 'created_at', 'updated_at', 'developer_key_id', 'user_request', 'render_time', 'user_agent', 'asset_user_access_id', 'participated', 'summarized', 'http_method', 'remote_ip', 'id', 'contributed', 'links', 'app_name'])#data[0].keys())  # header row
for row in data:
    output.writerow(row.values()) #values row
    



data = []
pages = 0
while pages!=3:
    next_url = [x['url'] for x in parsed_links if x['rel']=='next']
    print("next url: ",next_url[0])
    response1 = requests.get(next_url[0],headers=headers)
    pp("page: " + str(pages))
    pp("url: "+ next_url[0])
    pp(response1.json())
    pp("\n")
    pages +=1
    data +=response1.json()
    parsed_links = requests.utils.parse_header_links(response.headers['Link'].rstrip('>').replace('>,<', ',<'))
    
pp(data)


{'Cache-Control': 'max-age=0, private, must-revalidate', 'Content-Encoding': 'gzip', 'Content-Type': 'application/json; charset=utf-8', 'Date': 'Tue, 11 Dec 2018 22:07:29 GMT', 'ETag': 'W/"12503c540e7dd4e57c8780d9a460b740-gzip"', 'Link': '<https://canvas.upenn.edu/api/v1/users/5482827/page_views?end_time=2018-12-10&start_time=2018-12-01&end_time=2018-12-10&start_time=2018-12-01&page=first&per_page=100>; rel="current",<https://canvas.upenn.edu/api/v1/users/5482827/page_views?end_time=2018-12-10&start_time=2018-12-01&end_time=2018-12-10&start_time=2018-12-01&page=bookmark:WyIyMDE4LTEyLTA2VDE0OjMwOjQzLjU2MC0wNTowMCIsIjhhMWJiMWM5LTljNjYtNGQ5MS05NmNmLWU2NDhkNjM3ZmQwMiJd&per_page=100>; rel="next",<https://canvas.upenn.edu/api/v1/users/5482827/page_views?end_time=2018-12-10&start_time=2018-12-01&end_time=2018-12-10&start_time=2018-12-01&page=first&per_page=100>; rel="first"', 'P3P': 'CP="None, see http://www.instructure.com/privacy-policy"', 'Server': 'Apache', 'Set-Cookie': '_csrf_token=T5tV

[
    {
        'action': 'show',
        'app_name': None,
        'asset_type': None,
        'asset_user_access_id': None,
        'context_type': None,
        'contributed': False,
        'controller': 'canvadoc_sessions',
        'created_at': '2018-12-10T01:31:05Z',
        'developer_key_id': None,
        'http_method': 'get',
        'id': 'fdfb1d56-530d-4aa6-8f15-6ae5d1c6971a',
        'interaction_seconds': None,
        'links': {
            'account': 96678,
            'asset': None,
            'context': None,
            'real_user': None,
            'user': 5482827,
        },
        'participated': False,
        'remote_ip': '165.123.224.26',
        'render_time': 0.098983,
        'session_id': '0ef02c6c407e27740fa945fe6e77c8ec',
        'summarized': None,
        'updated_at': '2018-12-10T01:31:05Z',
        'url': 'https://canvas.upenn.edu/api/v1/canvadoc_session?blob=%7B%22anonymous_instructor_annotations%22:null,%22enable_annotations%22:null,%22moderated

'\ndata = []\npages = 0\nwhile pages!=3:\n    next_url = [x[\'url\'] for x in parsed_links if x[\'rel\']==\'next\']\n    print("next url: ",next_url[0])\n    response1 = requests.get(next_url[0],headers=headers)\n    pp("page: " + str(pages))\n    pp("url: "+ next_url[0])\n    pp(response1.json())\n    pp("\n")\n    pages +=1\n    data +=response1.json()\n    parsed_links = requests.utils.parse_header_links(response.headers[\'Link\'].rstrip(\'>\').replace(\'>,<\', \',<\'))\n    \npp(data)\n'

In [ ]:
#/api/v1/courses/:course_id/activity_stream/summary

current_course_ids = [ x['id'] for x in current_term]

student_id = '5389365' # confirmed correct id

for course_id in current_course_ids:
    url = '%s/api/v1/courses/%s/recent_students' % (domain,course_id)
    response = requests.get(url, headers=headers)
    r = response.json()


    display(HTML('<p><b>queried:</b> %s</p><p><b>result(%s):</b></p>' % (url,len(r))))
    pp(r)


In [ ]:


url = 'https://upenn.test.instructure.com/api/v1/users/5389365/page_views?end_time=2018-11-22&start_time=2018-11-20&page=first&per_page=100'
response = requests.get(url,headers=headers)
r= response.json()
pp(len(r))
pp(r)

In [ ]:
import csv,sys
outputFile = 'Student_Activity2.csv'
data = r
pp(data[0].keys())
output = csv.writer(open(outputFile,"w", newline='')) #create a csv.write
output.writerow(['session_id', 'url', 'context_type', 'asset_type', 'controller', 'action', 'interaction_seconds', 'created_at', 'updated_at', 'developer_key_id', 'user_request', 'render_time', 'user_agent', 'asset_user_access_id', 'participated', 'summarized', 'http_method', 'remote_ip', 'id', 'contributed', 'links', 'app_name'])#data[0].keys())  # header row
for row in data:
    output.writerow(row.values()) #values row
    

In [ ]:
pp(url)
parsed_links = requests.utils.parse_header_links(response.headers['Link'].rstrip('>').replace('>,<', ',<'))
pp(parsed_links)
